<a href="https://colab.research.google.com/github/yangxuan8/CMI-EVSI/blob/main/CMI_EVSI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Connect to Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import sys
sys.path.append('/content/drive/MyDrive')

Mounted at /content/drive


# Standard imports

In [ ]:
!pip install pytorch_lightning
import torch
import pickle
import argparse
import numpy as np
import torch.nn as nn
from torchmetrics import AUROC, Accuracy
from torch.utils.data import DataLoader, random_split, TensorDataset, ConcatDataset
import pytorch_lightning as pl
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.loggers import TensorBoardLogger
import pandas as pd
sys.path.append('/content/drive/MyDrive/DIME_main/experiments/hev')
import feature_groups
sys.path.append('/content/drive/MyDrive/DIME_main')
import dime
from dime.data_utils import HEVDataset, get_group_matrix, get_xy
from dime import MaskingPretrainer, CMIEstimator
from dime.utils import StaticMaskLayer1d, ConcreteMask, get_confidence, MaskLayerGrouped, get_mlp_network
import torch.optim as optim
from tqdm import tqdm
from baseline_models.base_model import BaseModel
sys.path.append('/content/drive/MyDrive/DIME_main/experiments')
from baselines import eddi, pvae, dfs
import time

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 801.6/801.6 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 840.4/840.4 kB 11.4 MB/s eta 0:00:00


# Set up command line arguments

In [ ]:
parser = argparse.ArgumentParser()
parser.add_argument('--gpu', type=int, default=0)
parser.add_argument('--method', type=str, default='dfs', choices=['eddi', 'dfs', 'apsa', 'fully_supervised'])
parser.add_argument('--use_feature_costs', default=False, action="store_true")
parser.add_argument('--num_trials', type=int, default=3)

_StoreAction(option_strings=['--num_trials'], dest='num_trials', nargs=None, const=None, default=3, type=<class 'int'>, choices=None, required=False, help=None, metavar=None)

# Load Dataset

Directly load the sampled dataset from csv without considering dataset splitting, for more details on obtaining the sample dataset, please refer to sample_dataset.ipynb

In [ ]:
hev_feature_names = feature_groups.hev_feature_names
hev_feature_groups = feature_groups.hev_feature_groups
auc_metric = AUROC(task='multiclass', num_classes=3)
acc_metric = Accuracy(task='multiclass', num_classes=3)

# Parse args
args = parser.parse_known_args()[0]
device = torch.device('cuda', args.gpu)
num_trials = args.num_trials
cols_to_drop = []
if cols_to_drop is not None:
    hev_feature_names = [item for item in hev_feature_names if str(hev_feature_names.index(item)) not in cols_to_drop]
# Load dataset
dataset = HEVDataset(data_dir=1, cols_to_drop=cols_to_drop)
d_in = dataset.X.shape[1]  # 32
d_out = len(np.unique(dataset.Y))  # 3

# Get features and groups
feature_groups_dict, feature_groups_mask = get_group_matrix(hev_feature_names, hev_feature_groups)
feature_group_indices = {i : key for i, key in enumerate(feature_groups_dict.keys())}
feat_to_ind = {key: i for i, key in enumerate(hev_feature_names)}

num_groups = len(feature_groups_mask)  # 32
print("Num groups=", num_groups)
print("Num features=", d_in)

# Split dataset
train_dataset, val_dataset, test_dataset = random_split(dataset, [0.8, 0.1, 0.1], generator=torch.Generator().manual_seed(0))
daataset_dict = dict(train_dataset=train_dataset, val_dataset=val_dataset, test_dataset=test_dataset)
f = open('/content/drive/MyDrive/dataset/data/dataset.pkl', "wb", pickle.HIGHEST_PROTOCOL)
pickle.dump(daataset_dict, f)

print(f'Train samples = {len(train_dataset)}, val samples = {len(val_dataset)}, test samples = {len(test_dataset)}')
# Find mean/variance for normalizing
x, y = get_xy(train_dataset)
mean = np.mean(x, axis=0)
std = np.clip(np.std(x, axis=0), 1e-3, None)

# Normalize via the original dataset
if args.method == 'eddi':
    dataset.X = (dataset.X - mean)/std
else:
    dataset.X = dataset.X - mean

# Set up data loaders.
train_dataloader = DataLoader(
    train_dataset, batch_size=128, shuffle=True, pin_memory=True,
    drop_last=True, num_workers=4)

val_dataloader = DataLoader(
    val_dataset, batch_size=1024, shuffle=False, pin_memory=True, drop_last=True, num_workers=4)

test_dataloader = DataLoader(
    test_dataset, batch_size=1024, shuffle=False, pin_memory=True, drop_last=True, num_workers=4)


mask_layer = MaskLayerGrouped(append=True, group_matrix=torch.tensor(feature_groups_mask))

Index(['faultNumber', 'VelocityRef:1', '<xdot>', '<BattSoc>', '<BattPwr>',
       '<Cltch1State>', '<Cltch2State>', '<BattV>', '<TransGear>', '<EngSpd>',
       '<IntkVlvLift>', '<EngTrq>', '<ThrPosPct>', '<WgAreaPct>',
       '<EgrVlvAreaPct>', '<VarCompRatioPos>', '<Acc>', '<Dec>', '<IgSw>',
       '<Chrg>', 'TransGear', 'BrkCmd', 'Cltch1Cmd', '<MotTrq>', '<StartTrq>',
       'StartCmd', 'MotTrqCmd', 'BattCrnt:1', 'MotPwrElec:1', 'MotPwrMech:1',
       'IntkVlvLiftCmd', 'FuelMainSoi', 'FuelFlw'],
      dtype='object')
Num groups= 32
Num features= 32
Train samples = 32902, val samples = 3656, test samples = 3655


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


# Set up networks

The original input is 2d_in, which is performed without considering the feature group. And right now we consider the Group features.

In [ ]:
hidden = 128
dropout = 0.3
predictor = nn.Sequential(
                      nn.Linear(d_in + num_groups, hidden),
                      nn.ReLU(),
                      nn.Dropout(dropout),
                      nn.Linear(hidden, hidden),
                      nn.ReLU(),
                      nn.Dropout(dropout),
                      nn.Linear(hidden, d_out))

value_network = nn.Sequential(
                      nn.Linear(d_in + num_groups, hidden),
                      nn.ReLU(),
                      nn.Dropout(dropout),
                      nn.Linear(hidden, hidden),
                      nn.ReLU(),
                      nn.Dropout(dropout),
                      nn.Linear(hidden, num_groups))

gdfs = dfs.GreedyDynamicSelection(selector, predictor, mask_layer).to(device)

# Training Process with different methods

1. EDDI: Efficient dynamic discovery of high-value information with partial VAE, a dynamic feature selection method that relies on a generative model
2. DFS: The simplest dynamic feature selection method provided considering the simple neural network
3. APSA: Our Access Point Search Algorithm, we consider non-uniform feature acquisition costs and combine the losses of the selector(value network) and predictor. For more details please refer to cmi_estimator.py
4. fully_supervised: simplest and fastest method

In [ ]:
num_features = [1, 3, 5, 10, 15, 20, 25, 30]
use_feature_costs = False
feature_costs = None
if args.use_feature_costs:
    feature_cost_df = pd.read_csv("data/feature_list_hev.csv")
    feature_costs = [feature_cost_df[feature_cost_df['Feature Name'] == feature]['Cost (Hours)'].item() for feature in list(feature_groups_dict.keys())]
    use_feature_costs = True

for trial in range(num_trials):

    results_dict = {
        'acc': {},
        'features': {}
    }


    if args.method == 'eddi':
        # Train PVAE.
        bottleneck = 16
        hidden = 128
        dropout = 0.3
        encoder = get_mlp_network(d_in + num_groups, bottleneck * 2)
        decoder = get_mlp_network(bottleneck, d_in)

        pv = pvae.PVAE(encoder, decoder, mask_layer, 20, 'gaussian').to(device)
        pv.fit(
            train_dataloader,
            val_dataloader,
            lr=1e-3,
            nepochs=10,
            verbose=False)

        # Train masked predictor.
        model = get_mlp_network(d_in + num_groups, d_out)
        sampler = None
        # if trial == 0:
        sampler = iterative.UniformSampler(get_xy(train_dataset)[0])  # TODO don't actually need sampler
        iterative_selector = iterative.IterativeSelector(model, mask_layer, sampler).to(device)
        iterative_selector.fit(
            train_dataloader,
            val_dataloader,
            lr=1e-3,
            nepochs=10,
            loss_fn=nn.CrossEntropyLoss(),
            patience=5,
            verbose=True)

        # Set up EDDI feature selection object.
        eddi_selector = eddi.EDDI(pv, model, mask_layer, feature_costs=feature_costs).to(device)

        # Evaluate.
        metrics_dict, cost_dict = eddi_selector.evaluate_multiple(test_dataloader, num_features, auc_metric, verbose=False)
        for num in num_features:
            acc = metrics_dict[num]
            results_dict['acc'][num] = acc
            print(f'Num = {num}, Acc = {100*acc:.2f}')

        print(results_dict)
        print(cost_dict)
        with open(f'/content/drive/MyDrive/dataset/results/hev_{args.method}_trial_{trial}_feature_costs_{use_feature_costs}.pkl', 'wb') as f:
            pickle.dump(results_dict, f)

        with open(f'/content/drive/MyDrive/dataset/results/hev_costs_{args.method}_trial_{trial}_feature_costs_{use_feature_costs}.pkl', 'wb') as f:
            pickle.dump(cost_dict, f)

    if args.method == 'dfs':
        max_features = 32

        # Prepare networks.
        predictor = get_mlp_network(d_in + num_groups, d_out)
        selector = get_mlp_network(d_in + num_groups, num_groups)

        # Pretrain predictor

        pretrain = MaskingPretrainer(
            predictor,
            mask_layer,
            lr=1e-3,
            loss_fn=nn.CrossEntropyLoss(),
            val_loss_fn=auc_metric)

        trainer = pl.Trainer(max_epochs=10)
        trainer.fit(pretrain, train_dataloader, val_dataloader)

        # Train selector and predictor jointly.
        gdfs = dfs.GreedyDynamicSelection(selector, predictor, mask_layer).to(device)
        gdfs.fit(
            train_dataloader,
            val_dataloader,
            lr=1e-3,
            nepochs=10,
            max_features=max_features,
            loss_fn=nn.CrossEntropyLoss(),
            patience=3,
            verbose=True)

        # Evaluate.
        for num in num_features:
            auroc_list = []
            acc_list = []

            auroc, acc = gdfs.evaluate(test_dataloader, num, (auc_metric, acc_metric))
            #results_dict['acc'][num] = acc
            #print(f'Num = {num}, Acc = {100*acc:.2f}')
            auroc_list.append(auroc)
            acc_list.append(acc)
            print(f'Num = {num}, AUROC = {100*auroc:.2f}, Acc = {100*acc:.2f}')

        with open(f'/content/drive/MyDrive/dataset/results/hev_{args.method}_trial_{trial}.pkl', 'wb') as f:
            pickle.dump(results_dict, f)

        # Save model
        gdfs.cpu()
        torch.save(gdfs, f'/content/drive/MyDrive/dataset/results/hev_{args.method}_trial_{trial}.pt')

    if args.method == 'apsa':
        max_features = 32
        start_time = time.time()

        # Prepare networks.
        hidden = 128
        dropout = 0.3
        predictor = nn.Sequential(
                  nn.Linear(d_in + num_groups, hidden),
                  nn.ReLU(),
                  nn.Dropout(dropout),
                  nn.Linear(hidden, hidden),
                  nn.ReLU(),
                  nn.Dropout(dropout),
                  nn.Linear(hidden, d_out))

        value_network = nn.Sequential(
                  nn.Linear(d_in + num_groups, hidden),
                  nn.ReLU(),
                  nn.Dropout(dropout),
                  nn.Linear(hidden, hidden),
                  nn.ReLU(),
                  nn.Dropout(dropout),
                  nn.Linear(hidden, num_groups))

        # Pretrain predictor
        pretrain = MaskingPretrainer(
            predictor,
            mask_layer,
            lr=1e-3,
            patience=6,
            loss_fn=nn.CrossEntropyLoss(),
            val_loss_fn=auc_metric
            )

        trainer = pl.Trainer(max_epochs=30)
        trainer.fit(pretrain, train_dataloader, val_dataloader)

        apsa = CMIEstimator(value_network,
                  predictor,
                  mask_layer,
                  lr=1e-3,
                  min_lr=1e-6,
                  max_features=30,
                  eps=0.1,
                  loss_fn=nn.CrossEntropyLoss(reduction='none'),
                  val_loss_fn=auc_metric,
                  eps_decay=0.2,
                  eps_steps=10,
                  patience=5,
                  feature_costs=feature_costs,
                  cmi_scaling='bounded')

        trainer = Trainer(accelerator='gpu',
                      devices=[args.gpu],
                      max_epochs=30,
                      precision=16,
                      logger=logger,
                      num_sanity_val_steps=0,
                      callbacks=[checkpoint_callback],
                      log_every_n_steps=10)

        trainer.fit(greedy_cmi_estimator, train_dataloader, val_dataloader)

        # Evaluate.
        for num in num_features:
            auroc_list = []
            acc_list = []
            auroc, acc = apsa.evaluate(test_dataloader, num, (auc_metric, acc_metric))
            auroc_list.append(auroc)
            acc_list.append(acc)
            print(f'Num = {num}, AUROC = {100*auroc:.2f}, Acc = {100*acc:.2f}')


        with open(f'/content/drive/MyDrive/dataset/results/hev_{args.method}_trial_{trial}.pkl', 'wb') as f:
            pickle.dump(results_dict, f)

        # Save model
        gdfs.cpu()
        torch.save(gdfs, f'/content/drive/MyDrive/dataset/results/hev_{args.method}_trial_{trial}.pt')

    # Train with full input
    if args.method == 'fully_supervised':
        model = get_mlp_network(d_in, d_out).to(device)
        opt = optim.Adam(model.parameters(), lr=1e-3)
        criterion = torch.nn.CrossEntropyLoss()
        scheduler = optim.lr_scheduler.ReduceLROnPlateau(
                opt, mode='min', factor=0.2, patience=5,
                min_lr=1e-5, verbose=True)

        num_bad_epochs = 0
        early_stopping_epochs = 6

        for epoch in range(20):
            model.train()
            train_batch_loss = 0
            val_batch_loss = 0
            val_pred_list = []
            val_y_list = []

            for i, (x, y) in enumerate(tqdm(train_dataloader)):
                x = x.to(device)
                y = y.to(device)

                pred = model(x)
                train_loss = criterion(pred, y)
                train_batch_loss += train_loss.item()
                train_loss.backward()
                opt.step()
                model.zero_grad()

            model.eval()

            with torch.no_grad():
                for i, (x, y) in enumerate(tqdm(val_dataloader)):
                    x = x.to(device)
                    y = y.to(device)

                    pred = model(x)
                    val_loss = criterion(pred, y)
                    val_batch_loss += val_loss.item()
                    val_pred_list.append(pred.cpu())
                    val_y_list.append(y.cpu())

                scheduler.step(val_batch_loss/len(val_dataloader))
                val_loss = val_batch_loss/len(val_dataloader)
                # Check if best model.
                if val_loss == scheduler.best:
                    # best_model = deepcopy(model)
                    num_bad_epochs = 0
                else:
                    num_bad_epochs += 1

                # Early stopping.
                if num_bad_epochs > early_stopping_epochs:
                    print(f'Stopping early at epoch {epoch+1}')
                    break

            print(f"Epoch: {epoch}, Train Loss: {train_batch_loss/len(train_dataloader)},"
                  + f"Val Loss: {val_batch_loss/len(val_dataloader)},"
                  + f"Val Performance: {auc_metric(torch.cat(val_pred_list), torch.cat(val_y_list))}")

        print("Evaluating on test set")

        model.eval()
        confidence_list = []
        test_pred_list = []
        test_y_list = []
        for i, (x, y) in enumerate(tqdm(test_dataloader)):
            x = x.to(device)
            y = y.to(device)

            pred = model(x)
            test_pred_list.append(pred.cpu())
            test_y_list.append(y.cpu())

            confidence_list.append(get_confidence(pred.cpu()))

        print(f"Test Performance:{auc_metric(torch.cat(test_pred_list), torch.cat(test_y_list))}")
        with open('/content/drive/MyDrive/dataset/confidence.npy', 'wb') as f:
            np.save(f, np.array(torch.cat(confidence_list).detach().numpy()))


In [ ]:
gdfs.selector.load_state_dict(torch.load('/content/drive/MyDrive/selPre/selector1'))
gdfs.predictor.load_state_dict(torch.load('/content/drive/MyDrive/selPre/predictor1'))

<All keys matched successfully>

# Evaluate performance

Performance on the test dataset

In [ ]:
# For saving results.
num_features = [1, 3, 5, 10, 15, 20, 25, 30]
auroc_list = []
acc_list = []

# Metrics (softmax is applied automatically in recent versions of torchmetrics).
auroc_metric = lambda pred, y: AUROC(task='multiclass', num_classes=3)(pred.softmax(dim=1), y)
acc_metric = Accuracy(task='multiclass', num_classes=3)

# Evaluate.
for num in num_features:
    auroc, acc = apsa.evaluate(test_dataloader, num, (auroc_metric, acc_metric))
    auroc_list.append(auroc)
    acc_list.append(acc)
    print(f'Num = {num}, AUROC = {100*auroc:.2f}, Acc = {100*acc:.2f}')

# Plot results

In [ ]:
fig, axarr = plt.subplots(1, 2, figsize=(18, 6))

# AUROC
plt.sca(axarr[0])
plt.plot(num_features, auroc_list, marker='o')
plt.xlabel('# Features', fontsize=16)
plt.ylabel('AUROC', fontsize=16)
plt.title('AUROC vs. Feature Budget', fontsize=18)
plt.tick_params(labelsize=14)
plt.gca().spines['right'].set_visible(False)
plt.gca().spines['top'].set_visible(False)

# Accuracy
plt.sca(axarr[1])
plt.plot(num_features, acc_list, marker='o')
plt.xlabel('# Features', fontsize=16)
plt.ylabel('Accuracy', fontsize=16)
plt.title('Accuracy vs. Feature Budget', fontsize=18)
plt.tick_params(labelsize=14)
plt.gca().spines['right'].set_visible(False)
plt.gca().spines['top'].set_visible(False)

plt.tight_layout()
plt.show()

# EVSI Functions

In [ ]:
cost_false_positive = 2
cost_false_negative = 4

x , y_train_tensor = get_xy(train_dataset)
#x_np=x.numpy()
x_train_df=pd.DataFrame.from_records(x)
y_train_tensor = torch.tensor(y_train_tensor)
unique_elements, counts = torch.unique(y_train_tensor, return_counts=True)
for i in range(unique_elements.size(0)):
  if unique_elements[i]==0:
    non_fault = counts[i]
    break
total = torch.sum(counts)

x , y_val_tensor = get_xy(val_dataset)
#x_np=x.numpy()
y_val_tensor = torch.tensor(y_val_tensor)
x_cv_df=pd.DataFrame.from_records(x)
unique_elements, counts = torch.unique(y_val_tensor, return_counts=True)
for i in range(unique_elements.size(0)):
  if unique_elements[i]==0:
    non_fault += counts[i]
total += torch.sum(counts)

P_present = non_fault/total
P_absent = 1 - P_present

In [ ]:
# helper function to calculate probability of correctly giving signal when present
def get_signal_present(prediction, ground_truth):
    present_index = list()
    for i in range(len(ground_truth)):
        if ground_truth[i] == 0:
            present_index.append(i)

    counter = 0
    for index in present_index:
        if prediction[index] == 0:
            counter += 1

    return counter/len(present_index)

# helper function to calculate probability of correctly giving signal when present
# there should be a more generic way using operator module to merge this with the one above.
def get_no_signal_absent(prediction, ground_truth):
    absent_index = list()
    for i in range(len(ground_truth)):
        if ground_truth[i] != 0:
            absent_index.append(i)

    counter = 0
    for index in absent_index:
        if prediction[index] != 0:
            counter += 1
    return counter/len(absent_index)

def get_expected_cost(prediction,GT_tensor):
  # get P(signal|present) and P(no signal|absent)
    ground_truth = GT_tensor.tolist()
    #prediction = prediction_t.tolist()
    P_signal_present = get_signal_present(prediction, ground_truth)
    P_no_signal_absent = get_no_signal_absent(prediction, ground_truth)
    P_signal_absent = 1 - P_no_signal_absent
    P_no_signal_present = 1 - P_signal_present

  # get P(signal)
    P_signal = P_present * P_signal_present + P_absent * P_signal_absent
    P_no_signal = 1 - P_signal

  # bayesian probability
    P_absent_signal = (P_signal_absent * P_absent) / P_signal
    P_present_signal = (P_signal_present * P_present) / P_signal
    P_absent_no_signal = (P_no_signal_absent * P_absent) / P_no_signal
    P_present_no_signal = (P_no_signal_present * P_present) / P_no_signal

  #calculate the evoi
    evoi = P_signal * min(cost_false_positive * P_absent_signal, cost_false_negative * P_present_signal) + P_no_signal * min(cost_false_positive * P_absent_no_signal, cost_false_negative * P_present_no_signal)

    return evoi

# Plot selections

In [ ]:
# Generate selections for entire test set.
num_features = range(1, 25)
p_list = []

for num in num_features:
  print(str(num)+'\n')
  x, y = get_xy(test_dataset)
  x = torch.tensor(x)
  x_np=x.numpy()
  x_test_df=pd.DataFrame(x)
  pred1, x_masked, m = gdfs(x.cuda(), max_features=num)
  pred=(pred1.cpu()).detach().numpy()
  pred = np.argmax(pred, axis=-1)
  p = m.mean(dim=0)
  p_list.append(p.cpu().numpy())

  sensors_index=np.array(p_list)
  sorted_indices = np.flip(np.argsort(sensors_index))
  sensors=[]
  for i in range(0,num):
    sensors = np.append(sensors, dataset.features[sorted_indices[0][i]])
  sensors=sensors.tolist()
  print(sensors)
  base_indices=sorted_indices[0][num:].tolist()
  sorted_indices=sorted_indices[0][0:num].tolist()
  print(get_expected_cost(pred,y))

1

['<BattSoc>']
tensor(1.1450)
2

['<BattSoc>', '<ThrPosPct>']
tensor(0.9068)
3

['<BattSoc>', '<BattV>', '<ThrPosPct>']
tensor(0.9322)
4

['<BattSoc>', '<BattV>', '<ThrPosPct>', '<WgAreaPct>']
tensor(0.8946)
5

['<BattSoc>', '<BattV>', '<ThrPosPct>', '<WgAreaPct>', '<IntkVlvLift>']
tensor(0.8813)
6

['<BattSoc>', '<BattV>', '<ThrPosPct>', '<WgAreaPct>', '<IntkVlvLift>', '<Cltch2State>']
tensor(0.9003)
7

['<BattSoc>', '<BattV>', '<IntkVlvLift>', '<ThrPosPct>', '<WgAreaPct>', '<EngSpd>', '<Cltch2State>']
tensor(0.9104)
8

['<Cltch2State>', '<WgAreaPct>', '<BattSoc>', '<ThrPosPct>', '<IntkVlvLift>', '<BattV>', '<EngSpd>', 'BattCrnt:1']
tensor(0.7839)
9

['<Cltch2State>', '<EngSpd>', '<WgAreaPct>', '<BattSoc>', '<ThrPosPct>', 'BattCrnt:1', '<IntkVlvLift>', '<BattV>', 'MotTrqCmd']
tensor(0.6943)
10

['<ThrPosPct>', '<EngSpd>', '<BattSoc>', 'BattCrnt:1', 'MotTrqCmd', '<IntkVlvLift>', '<WgAreaPct>', '<Cltch2State>', '<BattV>', 'IntkVlvLiftCmd']
tensor(0.6732)
11

['<WgAreaPct>', '<BattV>',